# Analiza regularyzacji w modelach MNIST

## Przebieg funkcji kosztu i dokładność

Model z Dropout ma **wyższy loss treningowy** (0.0619 vs 0.0162), ale **niższy loss walidacyjny** (0.0705 vs 0.0821). Dokładność testowa jest lepsza w modelu z regularyzacją (98.03% vs 97.75%), mimo niższej dokładności treningowej.

## Który model lepiej uogólnia?

**Model B z Dropout** lepiej uogólnia dane testowe - osiąga wyższą dokładność testową i mniejszą różnicę między wynikami train/validation, co wskazuje na mniejsze przeuczenie.

## Wpływ na szybkość zbieżności

Regularyzacja **spowolniła zbieżność** - Model B startuje z niższą dokładnością (90.66% vs 92.55% w epoch 1) i wolniej się uczy. Potrzebuje więcej epok, aby osiągnąć porównywalne wyniki treningowe.

## Najskuteczniejsze wartości parametrów

**Dropout = 0.3** okazał się optymalny, osiągając najlepszą dokładność testową (98.03%). Niższe wartości (0.2) są za słabe i prowadzą do przeuczenia, a wyższe (0.5) powodują underfitting i gorsze wyniki. 

In [1]:
import tensorflow as tf
import datetime

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


log_dir_a = "logs/fit/model_A_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir_b = "logs/fit/model_B_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback_a = tf.keras.callbacks.TensorBoard(log_dir=log_dir_a, histogram_freq=1)
tensorboard_callback_b = tf.keras.callbacks.TensorBoard(log_dir=log_dir_b, histogram_freq=1)

model_a = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model_a.compile(optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])


model_a.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test), callbacks=[tensorboard_callback_a])



model_b = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10, activation='softmax')
])

model_b.compile(optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

model_b.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test), callbacks=[tensorboard_callback_b])

print(f"Model A: {model_a.evaluate(x_test, y_test)}")
print(f"Model B: {model_b.evaluate(x_test, y_test)}")


/home/mateusz/.pyenv/versions/3.11.9/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1760984562.970769   57289 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5556 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10
  41/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4463 - loss: 1.7539

I0000 00:00:1760984565.257878   57596 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9255 - loss: 0.2591 - val_accuracy: 0.9582 - val_loss: 0.1379
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9655 - loss: 0.1153 - val_accuracy: 0.9700 - val_loss: 0.0991
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9759 - loss: 0.0791 - val_accuracy: 0.9699 - val_loss: 0.0917
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9818 - loss: 0.0592 - val_accuracy: 0.9720 - val_loss: 0.0862
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9861 - loss: 0.0455 - val_accuracy: 0.9758 - val_loss: 0.0811
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9891 - loss: 0.0353 - val_accuracy: 0.9730 - val_loss: 0.0880
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9915 - loss: 0.0275 - val_accuracy: 0.9793 - val_loss: 0.0720
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9929 - loss: 0.0228 - val_accurac

In [ ]:
# Wyświetlenie TensorBoard
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir logs/fit --port 6007

Reusing TensorBoard on port 6007 (pid 62567), started 0:00:15 ago. (Use '!kill 62567' to kill it.)